## CNN

필터의 size는 마음대로 하면 된다

```
w = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
```
=>  
tensorflow에는 쉽게 사용할 수 있는 함수가 있다

```
tf.nn.conv2d(X, w, strides=[1, 1, 1, 1], padding='SAME')
```
strides = [1, 옆으로 몇 칸씩, 밑으로 몇 칸씩, 1]  
padding은 숫자로 주지 않고 'SAME' 과 'VALID'가 있다.  
'SAME' 원래 이미지와 같은 크기의 Activation Map을 만들어 주겠다는 의미

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

In [3]:
X = tf.placeholder("float", [None, 784]) # mnist data image of shape 28*28=784
Y = tf.placeholder("float", [None, 10]) # 0-9 digits recognition => 10 classes

In [4]:
def init_weight(conv_fileter_array):
    return tf.Variable(tf.random_normal(conv_fileter_array, stddev=0.01))

In [5]:
w = init_weight([3,3,1,32]) # 3x3x1 conv, 32 ouputs
w2 = init_weight([3,3,32,64]) # 3x3x32 conv, 64 outputs
w3 = init_weight([3,3,64,128]) # 3x3x54 conv, 128 outputs

In [6]:
X_image = tf.reshape(X, [-1, 28, 28, 1]) # -1 : n개의 입력, 28 x 28, color code : 1

In [7]:
def relu_conv2d(x, w, strides):
    return tf.nn.relu(tf.nn.conv2d(x, w, strides=strides, padding='SAME'))

In [8]:
stride1 = [1, 1, 1, 1]
stride2 = [1, 2, 2, 1]

p_keep_conv = tf.placeholder('float')
p_keep_hidden = tf.placeholder("float")

l1a = relu_conv2d(X_image, w, stride1) # l1a shape=(?, 28, 28, 32)
l1 = tf.nn.max_pool(l1a, ksize=stride2, strides=stride2, padding='SAME') # l1 shape=(?, 14, 14, 32)
l1 = tf.nn.dropout(l1, p_keep_conv)

l2a = relu_conv2d(l1, w2, stride1) # l2a shape=(?, 14, 14, 64)
l2 = tf.nn.max_pool(l2a, ksize=stride2, strides=stride2, padding='SAME') # l2 shape=(?, 7, 7, 64)
l2 = tf.nn.dropout(l2, p_keep_conv)

l3a = relu_conv2d(l2, w3, stride1) # l3a shape=(?, 7, 7, 128)
l3 = tf.nn.max_pool(l3a, ksize=stride2, strides=stride2, padding='SAME') # l3 shape=(?, 4, 4, 128)

## Fully connected net

In [9]:
w4 = init_weight([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weight([625, 10]) # FC 625 inputs, 10 outputs

l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]]) # 펴주는 역할, reshape to (?, 2048)
l3 = tf.nn.dropout(l3, p_keep_conv)

l4 = tf.nn.relu(tf.matmul(l3, w4))
l4 = tf.nn.dropout(l4, p_keep_hidden)
py_x = tf.matmul(l4, w_o) # y bar

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(learning_rate, 0.9).minimize(cost)

 ### use another optimizer  
 
* GradientDescent  
* Adadelta  
* Adagrad  
* Momentum  
* Adam  
* Ftrl  
* RMSP

In [11]:
mnist = input_data.read_data_sets("train_data/MNIST_data/", one_hot=True)

Extracting train_data/MNIST_data/train-images-idx3-ubyte.gz
Extracting train_data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting train_data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting train_data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

In [13]:
# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0;
    total_batch = int(mnist.train.num_examples/batch_size)

    # Loop over all batches
    for i in range(total_batch) :
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # Fit training using batch data
        sess.run(train_op, feed_dict = {X: batch_xs, Y: batch_ys, p_keep_conv: 0.8, p_keep_hidden: 0.5})
        avg_cost += sess.run(cost, feed_dict = {X: batch_xs, Y: batch_ys, p_keep_conv: 0.8, p_keep_hidden: 0.5})/total_batch

    if epoch % display_step == 0:
        print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

print ("Optimization Finished!")

Epoch: 0001 cost= 0.993453501
Epoch: 0002 cost= 0.059542507
Epoch: 0003 cost= 0.035192681
Epoch: 0004 cost= 0.025442494
Epoch: 0005 cost= 0.022640701
Epoch: 0006 cost= 0.019629148
Epoch: 0007 cost= 0.018035898
Epoch: 0008 cost= 0.015763352
Epoch: 0009 cost= 0.013936277
Epoch: 0010 cost= 0.014627520
Epoch: 0011 cost= 0.014389292
Epoch: 0012 cost= 0.013688243
Epoch: 0013 cost= 0.014286589
Epoch: 0014 cost= 0.013022557
Epoch: 0015 cost= 0.012090100
Optimization Finished!


In [14]:
# Test model
correct_prediction = tf.equal(tf.argmax(py_x, 1), tf.argmax(Y, 1))
# Calcuate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print ("Accuracy: ", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels, p_keep_conv: 1.0, p_keep_hidden: 1.0}, session=sess))

Accuracy:  0.9937
